In [2]:
import pandas as pd
import json
import os


from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [3]:
load_dotenv()

api_key = os.getenv('API_KEY')

df = pd.read_csv("../data/Fine-tuning Data_raw.csv").drop(columns="텍스트 - ERD 생성 데이터")
df.head()

,인덱스,텍스트 - 요약/정리 데이터
0,1,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹..."
1,2,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2,3,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로..."
3,4,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있..."
4,5,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안..."


In [4]:
dataset = df.copy().drop(columns= '인덱스')
dataset.columns = ['text']
dataset

,text
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안..."
...,...
95,"지역 축제의 혼잡도를 낮추고 운영 효율을 높이기 위해, 관람객의 이동 경로와 체류 ..."
96,작물의 생장 조건을 분석하고 환경을 자동으로 조절하는 스마트팜 제어 시스템을 기획하...
97,청소년의 감정 상태를 분석하고 적절한 상담 자원을 연결해주는 정서 관리 플랫폼을 기...
98,가정에서 발생하는 음식물 쓰레기를 줄이기 위한 스마트 분리 수거 시스템을 기획하였다...


In [ ]:
def contents_data(df):

  contents_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 체계적으로 분석하고 요약해주는 프로그램입니다.
  주어진 텍스트를 핵심 요소별로 정리하여 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
    "project_summary": {{
      "title": "사용자 관리 플랫폼",
      "category": "웹 애플리케이션",
      "platform_type": "웹/모바일",
      "target_users": [
        "일반 사용자",
        "관리자",
        "기업 고객"
      ],
      "main_purpose": "효율적인 사용자 관리 및 데이터 분석을 통한 비즈니스 최적화",
      "key_features": [
        {{
          "feature": "사용자 등록 및 인증",
          "description": "안전한 회원가입, 로그인, 비밀번호 관리 기능"
        }},
        {{
          "feature": "대시보드",
          "description": "사용자 활동 통계 및 데이터 시각화"
        }},
        {{
          "feature": "권한 관리",
          "description": "역할 기반 접근 제어 시스템"
        }}
      ],
      "core_technologies": [
        {{
          "category": "Frontend",
          "technologies": ["React", "TypeScript", "Tailwind CSS"]
        }},
        {{
          "category": "Backend", 
          "technologies": ["Node.js", "Express", "JWT"]
        }},
        {{
          "category": "Database",
          "technologies": ["PostgreSQL", "Redis"]
        }},
        {{
          "category": "Infrastructure",
          "technologies": ["AWS", "Docker", "Nginx"]
        }}
      ],
      "problem_solving": {{
        "current_problem": "기존 사용자 관리 시스템의 비효율성과 보안 취약점",
        "solution_approach": "모던 웹 기술과 AI 기반 분석을 통한 통합 관리 플랫폼 구축",
        "expected_benefits": [
          "관리 효율성 50% 향상",
          "보안 강화를 통한 데이터 보호",
          "사용자 경험 개선"
        ]
      }},
      "special_features": [
        "AI 기반 사용자 행동 분석",
        "실시간 알림 시스템",
        "다국어 지원",
        "모바일 반응형 디자인"
      ],
      "business_model": {{
        "type": "SaaS",
        "revenue_streams": [
          "월 구독료",
          "프리미엄 기능",
          "기업용 라이선스"
        ],
        "target_market": "중소기업 및 스타트업"
      }},
      "scalability": {{
        "user_capacity": "최대 10만명 동시 접속",
        "expansion_plan": "글로벌 서비스 확장",
        "integration_capability": "타사 시스템 연동 API 제공"
      }},
      "development_timeline": {{
        "estimated_duration": "6개월",
        "key_milestones": [
          {{
            "phase": "기획 및 설계",
            "duration": "1개월"
          }},
          {{
            "phase": "MVP 개발",
            "duration": "3개월"
          }},
          {{
            "phase": "테스트 및 배포",
            "duration": "2개월"
          }}
        ]
      }},
      "success_metrics": [
        "사용자 증가율",
        "시스템 응답 시간",
        "사용자 만족도",
        "보안 사고 발생률"
      ],
      "challenges_and_risks": [
        {{
          "challenge": "데이터 보안",
          "mitigation": "다층 보안 체계 구축"
        }},
        {{
          "challenge": "확장성",
          "mitigation": "마이크로서비스 아키텍처 적용"
        }}
      ]
    }}
  }}

  다음 항목들을 반드시 포함하여 요약하세요:
  - 프로젝트 제목 및 카테고리
  - 타겟 사용자 및 주요 목적
  - 핵심 기능들과 상세 설명
  - 사용 기술 스택 (명시된 경우)
  - 해결하고자 하는 문제와 솔루션
  - 특별한 기능이나 차별점
  - 비즈니스 모델 (추론 가능한 경우)
  - 확장성 및 향후 계획
  - 예상 개발 일정 (추정)
  - 성공 지표 및 도전 과제

  텍스트에서 명시되지 않은 부분은 프로젝트 성격을 고려하여 합리적으로 추론하여 작성하세요.
  각 항목별로 구체적이고 실용적인 정보를 제공하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    data = json.loads(answer.content)
    contents_lst.append(data)
    print(f'내용 요약의 {i}번 진행중입니다.')
  return contents_lst 

def API_specs_data(df):

  API_lst = []

  sys_template = """
  당신은 사용자가 입력한 프로젝트 개요를 기반으로 예상되는 API 명세서를 작성해주는 프로그램입니다.
  주제에 대해서 예상되는 RESTful API 명세서를 JSON 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
  "api_specification": {{
      "openapi": "3.0.0",
      "info": {{
      "title": "User Management API",
      "version": "1.0.0",
      "description": "사용자 관리를 위한 API"
      }},
      "servers": [
      {{
          "url": "https://api.example.com/v1",
          "description": "Production server"
      }}
      ],
      "paths": {{
      "/users": {{
          "get": {{
          "summary": "사용자 목록 조회",
          "description": "등록된 모든 사용자 목록을 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "page",
              "in": "query",
              "description": "페이지 번호",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 1
              }}
              }},
              {{
              "name": "limit",
              "in": "query", 
              "description": "페이지당 항목 수",
              "required": false,
              "schema": {{
                  "type": "integer",
                  "default": 10
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 목록 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "type": "array",
                          "items": {{
                          "$ref": "#/components/schemas/User"
                          }}
                      }},
                      "pagination": {{
                          "$ref": "#/components/schemas/Pagination"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "post": {{
          "summary": "새 사용자 생성",
          "description": "새로운 사용자를 생성합니다",
          "tags": ["Users"],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/CreateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "201": {{
              "description": "사용자 생성 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "400": {{
              "description": "잘못된 요청 데이터",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }},
      "/users/{{userId}}": {{
          "get": {{
          "summary": "특정 사용자 조회",
          "description": "사용자 ID로 특정 사용자 정보를 조회합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 조회 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "put": {{
          "summary": "사용자 정보 수정",
          "description": "사용자 정보를 수정합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "requestBody": {{
              "required": true,
              "content": {{
              "application/json": {{
                  "schema": {{
                  "$ref": "#/components/schemas/UpdateUserRequest"
                  }}
              }}
              }}
          }},
          "responses": {{
              "200": {{
              "description": "사용자 정보 수정 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "data": {{
                          "$ref": "#/components/schemas/User"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }},
          "delete": {{
          "summary": "사용자 삭제",
          "description": "사용자를 삭제합니다",
          "tags": ["Users"],
          "parameters": [
              {{
              "name": "userId",
              "in": "path",
              "required": true,
              "description": "사용자 ID",
              "schema": {{
                  "type": "integer"
              }}
              }}
          ],
          "responses": {{
              "200": {{
              "description": "사용자 삭제 성공",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "type": "object",
                      "properties": {{
                      "status": {{
                          "type": "string",
                          "example": "success"
                      }},
                      "message": {{
                          "type": "string",
                          "example": "사용자가 성공적으로 삭제되었습니다"
                      }}
                      }}
                  }}
                  }}
              }}
              }},
              "404": {{
              "description": "사용자를 찾을 수 없음",
              "content": {{
                  "application/json": {{
                  "schema": {{
                      "$ref": "#/components/schemas/Error"
                  }}
                  }}
              }}
              }}
          }}
          }}
      }}
      }},
      "components": {{
      "schemas": {{
          "User": {{
          "type": "object",
          "properties": {{
              "id": {{
              "type": "integer",
              "example": 1
              }},
              "username": {{
              "type": "string",
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "createdAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }},
              "updatedAt": {{
              "type": "string",
              "format": "date-time",
              "example": "2024-01-15T10:30:00Z"
              }}
          }}
          }},
          "CreateUserRequest": {{
          "type": "object",
          "required": ["username", "email"],
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john@example.com"
              }},
              "password": {{
              "type": "string",
              "minLength": 8,
              "example": "password123"
              }}
          }}
          }},
          "UpdateUserRequest": {{
          "type": "object",
          "properties": {{
              "username": {{
              "type": "string",
              "minLength": 3,
              "maxLength": 50,
              "example": "john_doe_updated"
              }},
              "email": {{
              "type": "string",
              "format": "email",
              "example": "john_updated@example.com"
              }}
          }}
          }},
          "Pagination": {{
          "type": "object",
          "properties": {{
              "currentPage": {{
              "type": "integer",
              "example": 1
              }},
              "totalPages": {{
              "type": "integer",
              "example": 10
              }},
              "totalItems": {{
              "type": "integer",
              "example": 100
              }},
              "itemsPerPage": {{
              "type": "integer",
              "example": 10
              }}
          }}
          }},
          "Error": {{
          "type": "object",
          "properties": {{
              "status": {{
              "type": "string",
              "example": "error"
              }},
              "message": {{
              "type": "string",
              "example": "오류가 발생했습니다"
              }},
              "code": {{
              "type": "string",
              "example": "VALIDATION_ERROR"
              }}
          }}
          }}
      }},
      "securitySchemes": {{
          "bearerAuth": {{
          "type": "http",
          "scheme": "bearer",
          "bearerFormat": "JWT"
          }}
      }}
      }},
      "security": [
      {{
          "bearerAuth": []
      }}
      ]
  }}
  }}

  각 API 엔드포인트의 상세 정보(HTTP 메서드, 경로, 요청/응답 스키마, 상태 코드)를 모두 포함하세요.
  프로젝트 특성에 맞는 추가적인 엔드포인트나 기능이 필요하면 포함하여 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  OpenAPI 3.0 표준을 따르며, 실제 구현 가능한 수준의 상세한 명세서를 작성하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    try :
      data = json.loads(answer.content)
      API_lst.append(data)
      print(f'API 명세서의 {i}번 진행중입니다.')
    except :
      print(f'{i}번쨰 오류발생')
      continue

  return API_lst
def ERD_data(df):

  entities_lst = []
  relationships_lst = []

  sys_template = """
  당신은 사용자가 입력한 문장의 개요를 예상 ERD를 작성해주는 프로그램입니다. 
  주제에 대해서 예상되는 ERD를 json 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  예시는 다음과 같습니다.
  {{
    "entities": [
      {{
        "name": "User",
        "attributes": [
          {{
            "name": "user_id",
            "data_type": "INTEGER",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "username",
            "data_type": "VARCHAR(50)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "email",
            "data_type": "VARCHAR(100)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": true
          }}
        ]
      }}
    ],
    "relationships": [
      {{
        "from": "User",
        "to": "Order",
        "type": "one-to-many",
        "foreign_key": "user_id",
        "constraint_name": "fk_order_user"
      }}
    ]
  }}

  각 속성의 상세 정보(PK, FK, NULL 허용 여부, 데이터 타입)를 모두 포함하세요.
  제한된 방법 이외의 더 포함할 수 있는 자료가 있으면 같이 작성하세요.
  JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=api_key,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    data = json.loads(answer.content)
    entities_lst.append(data['entities'])
    relationships_lst.append(data['relationships'])
    print(f'ERD의 {i}번 진행중입니다.')
    
  return entities_lst, relationships_lst 

In [9]:
dataset['contents'] = contents_lst
dataset.to_csv("../data/contents_data.csv")

In [ ]:
entities_lst, relationships_lst = ERD_data(df)
dataset['relationships_data'] = relationships_lst
dataset['ERD_data'] = entities_lst
dataset.to_csv("../data/ERD_data.csv")

ERD의 0번 진행중입니다.
ERD의 1번 진행중입니다.
ERD의 2번 진행중입니다.
ERD의 3번 진행중입니다.
ERD의 4번 진행중입니다.
ERD의 5번 진행중입니다.
ERD의 6번 진행중입니다.
ERD의 7번 진행중입니다.
ERD의 8번 진행중입니다.
ERD의 9번 진행중입니다.
ERD의 10번 진행중입니다.
ERD의 11번 진행중입니다.
ERD의 12번 진행중입니다.
ERD의 13번 진행중입니다.
ERD의 14번 진행중입니다.
ERD의 15번 진행중입니다.
ERD의 16번 진행중입니다.
ERD의 17번 진행중입니다.
ERD의 18번 진행중입니다.
ERD의 19번 진행중입니다.
ERD의 20번 진행중입니다.
ERD의 21번 진행중입니다.
ERD의 22번 진행중입니다.
ERD의 23번 진행중입니다.
ERD의 24번 진행중입니다.
ERD의 25번 진행중입니다.
ERD의 26번 진행중입니다.
ERD의 27번 진행중입니다.
ERD의 28번 진행중입니다.
ERD의 29번 진행중입니다.
ERD의 30번 진행중입니다.
ERD의 31번 진행중입니다.
ERD의 32번 진행중입니다.
ERD의 33번 진행중입니다.
ERD의 34번 진행중입니다.
ERD의 35번 진행중입니다.
ERD의 36번 진행중입니다.
ERD의 37번 진행중입니다.
ERD의 38번 진행중입니다.
ERD의 39번 진행중입니다.
ERD의 40번 진행중입니다.
ERD의 41번 진행중입니다.
ERD의 42번 진행중입니다.
ERD의 43번 진행중입니다.
ERD의 44번 진행중입니다.
ERD의 45번 진행중입니다.
ERD의 46번 진행중입니다.
ERD의 47번 진행중입니다.
ERD의 48번 진행중입니다.
ERD의 49번 진행중입니다.
ERD의 50번 진행중입니다.
ERD의 51번 진행중입니다.
ERD의 52번 진행중입니다.
ERD의 53번 진행중입니다.
ERD의 54번 진행중입니다.
ERD의 55번 진행중입니다.
ERD의 56번 진행중입니다.
ERD의 57번 진행중입니다.
ERD의 58번 진행중입니다.
ERD의 59

In [ ]:
api_specs_lst = API_specs_data(df)
dataset['API_specs_data'] = api_specs_lst
dataset.to_csv("../data/api_data.csv")

In [ ]:
contents_lst = contents_data(df)
api_specs_lst = API_specs_data(df)
entities_lst, relationships_lst = ERD_data(df)

dataset['contents'] = contents_lst
dataset['ERD_data'] = entities_lst
dataset['relationships_data'] = relationships_lst


dataset.to_csv("../data/contents_data.csv")

내용 요약의 0번 진행중입니다.
내용 요약의 1번 진행중입니다.
내용 요약의 2번 진행중입니다.
내용 요약의 3번 진행중입니다.
내용 요약의 4번 진행중입니다.
내용 요약의 5번 진행중입니다.
내용 요약의 6번 진행중입니다.
내용 요약의 7번 진행중입니다.
내용 요약의 8번 진행중입니다.
내용 요약의 9번 진행중입니다.
내용 요약의 10번 진행중입니다.
내용 요약의 11번 진행중입니다.
내용 요약의 12번 진행중입니다.
내용 요약의 13번 진행중입니다.
내용 요약의 14번 진행중입니다.
내용 요약의 15번 진행중입니다.
내용 요약의 16번 진행중입니다.
내용 요약의 17번 진행중입니다.
내용 요약의 18번 진행중입니다.
내용 요약의 19번 진행중입니다.
내용 요약의 20번 진행중입니다.
내용 요약의 21번 진행중입니다.
내용 요약의 22번 진행중입니다.
내용 요약의 23번 진행중입니다.
내용 요약의 24번 진행중입니다.
내용 요약의 25번 진행중입니다.
내용 요약의 26번 진행중입니다.
내용 요약의 27번 진행중입니다.
내용 요약의 28번 진행중입니다.
내용 요약의 29번 진행중입니다.
내용 요약의 30번 진행중입니다.
내용 요약의 31번 진행중입니다.
내용 요약의 32번 진행중입니다.
내용 요약의 33번 진행중입니다.
내용 요약의 34번 진행중입니다.
내용 요약의 35번 진행중입니다.
내용 요약의 36번 진행중입니다.
내용 요약의 37번 진행중입니다.
내용 요약의 38번 진행중입니다.
내용 요약의 39번 진행중입니다.
내용 요약의 40번 진행중입니다.
내용 요약의 41번 진행중입니다.
내용 요약의 42번 진행중입니다.
내용 요약의 43번 진행중입니다.
내용 요약의 44번 진행중입니다.
내용 요약의 45번 진행중입니다.
내용 요약의 46번 진행중입니다.
내용 요약의 47번 진행중입니다.
내용 요약의 48번 진행중입니다.
내용 요약의 49번 진행중입니다.
내용 요약의 50번 진행중입니다.
내용 요약의 51번 진행중입니다.
내용 요약의 52번 진행중입니다.
내용 

JSONDecodeError: Expecting property name enclosed in double quotes: line 164 column 19 (char 4399)